In [3]:
#!pip install EIA_python
import numpy as np 
import pandas as pd
import json
import eia

In [4]:
apiKey = "5f54b3e66477e22ec068066b1de8026d"
api = eia.API(apiKey)

In [ ]:
url = "http://api.eia.gov/geoset/?geoset_id=sssssss&regions=region1,region2,region3,...&api_key={0}[&start=|&num=][&end=][&out=xml|json]".format(EIAapiKey)

In [5]:
series_id_list = ["INTL.57-1-DZA-TBPD.M", "INTL.57-1-AGO-TBPD.M", "INTL.57-1-COG-TBPD.M", "INTL.57-1-COD-TBPD.M", 
                  "INTL.57-1-ECU-TBPD.M","INTL.57-1-GNQ-TBPD.M","INTL.57-1-GAB-TBPD.M","INTL.57-1-IRN-TBPD.M",
                  "INTL.57-1-IRQ-TBPD.M", "INTL.57-1-KWT-TBPD.M", "INTL.57-1-LBY-TBPD.M", "INTL.57-1-NGA-TBPD.M",
                  "INTL.57-1-QAT-TBPD.M", "INTL.57-1-RUS-TBPD.M", "INTL.57-1-SAU-TBPD.M","INTL.57-1-ARE-TBPD.M",
                  "INTL.57-1-VEN-TBPD.M"]

In [6]:
df_list = [pd.DataFrame(api.data_by_series(series)) for series in series_id_list]

In [27]:
oil_data = pd.concat(df_list, axis=1)

In [31]:
oil_data = oil_data.replace("--", np.nan)

In [34]:
oil_data.dropna().shape

(330, 17)

In [36]:
oil_data.shape

(570, 17)

In [39]:
oil_data_reduced = oil_data.dropna()

In [ ]:
oil_data_reduced